# Movie Preprocessor
### Preprocesses Movies from IDR downloader

#### Import libraries

In [ ]:

#pip install pybasic not working so must use this
import PyBaSiC.pybasic as pybasic
from matplotlib import pyplot as plt
from PIL import Image
import os
from PIL import *
import numpy as np
from skimage.io import imsave
import imagej
from IPython.utils.io import capture_output

#### Load Movie Data as List of Numpy Arrays

In [ ]:
# Load function for webp movies downloaded with 0.download_data/IDR_downloader.ipynb
def load_webp_data(path):
    images = []
    for filename in sorted(os.listdir(path)):
        if filename == "annotations":
            continue
        f = os.path.join(path, filename)
        image = Image.open(f)
        image = image.convert("L")
        images.append(np.array(image))
    return images

# Load function for ch5 movies downloaded with 0.download_data/aspera_IDR_downloader.ipynb
def load_ch5_data(ij, path):
    with capture_output() as captured:
        jimage = ij.io().open(path)
        image = ij.py.from_java(jimage)
        #only get last 94 frames of movies (at least one movie has 95)
        im_arr = image.values[-94:, :, :, 0]
        return im_arr

#### PyBaSiC Illumination correction as described in http://www.nature.com/articles/ncomms14836

In [ ]:
def pybasic_illumination_correction(brightfield_images):
    
    flatfield, darkfield = pybasic.basic(brightfield_images, darkfield=True)
    
    baseflour = pybasic.background_timelapse(
        images_list = brightfield_images, 
        flatfield = flatfield, 
        darkfield = darkfield
    )
    
    brightfield_images_corrected_original = pybasic.correct_illumination(
        images_list = brightfield_images, 
        flatfield = flatfield, 
        darkfield = darkfield,
        background_timelapse = baseflour
    )
    
    #convert corrected images to numpy array, normalize, and convert to uint8
    brightfield_images_corrected = np.array(brightfield_images_corrected_original)
    brightfield_images_corrected[brightfield_images_corrected<0] = 0 #make negatives 0
    brightfield_images_corrected = brightfield_images_corrected / np.max(brightfield_images_corrected) # normalize the data to 0 - 1
    brightfield_images_corrected = 255 * brightfield_images_corrected # Now scale by 255
    corrected_movie = brightfield_images_corrected.astype(np.uint8)
    
    return corrected_movie

#### Preprocess movies in training data

In [ ]:
def preprocess_labeled_data(ij, load_path, save_path):
    for plate in os.listdir(load_path):
        for well in os.listdir(load_path + "/" + plate):
            print("Processing plate " + plate + " well " + well)
            well_load_path = load_path + "/" + plate + "/" + well
            new_well_name = well.replace("_01.ch5", ".tiff")
            
            #check if well has already been preprocessed
            if os.path.isdir(save_path + "/" + plate + "/"):
                print("This movie has already been preprocessed")
                continue
                
            well_save_path = save_path + "/" + plate + "/" + new_well_name
            original_movie = load_ch5_data(ij, well_load_path)
            corrected_movie = pybasic_illumination_correction(original_movie)
            os.makedirs(save_path + "/" + plate + "/")
            imsave(well_save_path, corrected_movie)
       
ij = imagej.init('/home/roshankern/Desktop/Fiji.app')
load_path = "/media/roshankern/hdd/mitocheck_idr_download/labled_movies"
save_path = "/media/roshankern/hdd/mitocheck_idr_download/labled_movies_preprocessed"
preprocess_labeled_data(ij, load_path, save_path)